In [29]:
gesture_type = 'two_up'

In [30]:
import json 
import os

img_SOURCE = f"./../datasets/HaGRID/{gesture_type}/"
all_images_of_given_gesture = [f for f in os.listdir(os.path.join(img_SOURCE)) if f.endswith('.jpg')]
all_images_of_given_gesture_set = set(os.path.splitext(f)[0] for f in all_images_of_given_gesture)

print("All Images")
print(len(all_images_of_given_gesture))
print(len(all_images_of_given_gesture_set))

train_labels_SOURCE = f"./../datasets/HaGRID/train/{gesture_type}.json"
test_labels_SOURCE = f"./../datasets/HaGRID/test/{gesture_type}.json"
val_labels_SOURCE = f"./../datasets/HaGRID/val/{gesture_type}.json"

train_labels = []
test_labels = []
val_labels = []

with open(train_labels_SOURCE, 'r') as f:
    data = json.load(f)
    for image_id, info in data.items():
        train_labels.append(image_id)
        
with open(test_labels_SOURCE, 'r') as f:
    data = json.load(f)
    for image_id, info in data.items():
        test_labels.append(image_id)
        
with open(val_labels_SOURCE, 'r') as f:
    data = json.load(f)
    for image_id, info in data.items():
        val_labels.append(image_id)

print("Train Labels")
train_labels_set = set(train_labels)
print(len(train_labels))
print(len(train_labels_set))
print(train_labels_set.issubset(all_images_of_given_gesture_set))

print("Test Labels")
test_labels_set = set(test_labels)
print(len(test_labels))
print(len(test_labels_set))
print(test_labels_set.issubset(all_images_of_given_gesture_set))

print("Val Labels")
val_labels_set = set(val_labels)
print(len(val_labels))
print(len(val_labels_set))
print(val_labels_set.issubset(all_images_of_given_gesture_set))

print("Labels Combined")
labels_set = train_labels_set | test_labels_set | val_labels_set
print(len(labels_set))
labels_set == all_images_of_given_gesture_set

All Images
30688
30688
Train Labels
22688
22688
True
Test Labels
5000
5000
True
Val Labels
3000
3000
True
Labels Combined
30688


True

In [39]:
set_type = 'val'

In [40]:
WEIGHTS = "./../training/runs/train/MIAP_person_detection/weights/best.pt"
SOURCE = f"./../datasets/HaGRID/{set_type}/{gesture_type}.json"

In [41]:
%run ./HaGRID_conversion_utils/detect.py --weights $WEIGHTS --conf-thres 0.317 \
--device '0' --img-size 640 --source $SOURCE

INFO:utils.torch_utils:YOLOR 🚀 893edf6 torch 2.3.0+cu121 CUDA:0 (NVIDIA A100 80GB PCIe, 81221.8125MB)

INFO:utils.torch_utils:Model Summary: 208 layers, 6007596 parameters, 0 gradients, 13.0 GFLOPS


Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

found 3000 image IDs in ./../datasets/HaGRID/val/two_up.json
ended up with 3000 IDs after converting to a set
list image path length = 3000


3000it [02:32, 19.61it/s]


In [42]:
import os

# Define the directories
images_dir = f'./../datasets/HaGRID_converted/images/{set_type}'
labels_dir = f'./../datasets/HaGRID_converted/labels/{set_type}'

# Get the list of filenames (without extensions) in both directories
image_filenames = {os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.endswith('.jpg')}
label_filenames = {os.path.splitext(f)[0] for f in os.listdir(labels_dir) if f.endswith('.txt')}

# Find mismatches
missing_in_labels = image_filenames - label_filenames
missing_in_images = label_filenames - image_filenames

# Print results
if not missing_in_labels and not missing_in_images:
    print("All filenames match!")
else:
    if missing_in_labels:
        print("Images without labels:", missing_in_labels)
    if missing_in_images:
        print("Labels without images:", missing_in_images)

print(len(image_filenames))
set(image_filenames) == set(label_filenames)

All filenames match!
12000


True

In [44]:
from os import listdir
from os.path import abspath, join, isdir, isfile

path = f"./../datasets/HaGRID_converted/"
print("The HaGRID_converted dataset is located in " + abspath(path) + ". Directory content:\n")

content = sorted(listdir(path))
directories = [directory for directory in content if isdir(join(path, directory))]

for directory in directories:
    if not directory.startswith('.'):
        print(directory)
        subcontent = sorted(listdir(join(path, directory)))
        for file in subcontent:
            subdirectory_path = join(path, directory, file)
            if isdir(subdirectory_path):
                num_files = len([f for f in listdir(subdirectory_path) if isfile(join(subdirectory_path, f))])
                print('├──' + file + f"    <-- contains {num_files} files") if not file.startswith('.') else None

The HaGRID_converted dataset is located in /home/jovyan/datasets/HaGRID_converted. Directory content:

images
├──test    <-- contains 20000 files
├──train    <-- contains 92795 files
├──val    <-- contains 12000 files
labels
├──test    <-- contains 20000 files
├──train    <-- contains 92795 files
├──val    <-- contains 12000 files
